In [5]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from label_to_cat import LABEL_TO_CAT
from probs_saver import ProbStore

datapath = '?'
save_to = '../input/resnet_val_comp.h5'

In [6]:
%%time
store = pd.HDFStore(datapath)
keys = store.keys()
print('{} tables in storage'.format(len(keys)))

0 tables in storage
CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 1.79 ms


In [ ]:
cols = []
for i in range(1, 5271):
    cols.append(i)
    
saver = ProbStore(path=save_to)

for key in tqdm(keys): 
    table = store.select(key)
    assert table.shape[0] % 10 == 0
    two_cols = []
    sums_df = pd.DataFrame(columns=cols, dtype=np.float16)
    for i in range(int(table.shape[0] / 10)):
        st = 10 * i
        product_id = table.pr_id.iloc[st]
        img_num = table.img_num.iloc[st]
        sum1 = table.iloc[pd.np.r_[st : st + 4, st + 5:st + 9]].drop(['pr_id', 'img_num'], axis=1, inplace=False).sum()
        sum2 = table.iloc[pd.np.r_[st + 4: st + 5, st + 9:st + 10]].drop(['pr_id', 'img_num'], axis=1, inplace=False).sum()
        sum_ = sum1 + sum2*3
        
        two_cols.append((product_id, img_num))
        sums_df.append(sum_, ignore_index=True)  
    two_cols_df = pd.DataFrame(data=two_cols, columns=['pr_id', 'img_num'], dtype=np.int32)
    df = pd.concat([two_cols_df, sums_df], axis=1, ignore_index=True)
    saver.saveProbs(df)
    # debug
    print(df)
    break


# debug

In [ ]:
%%time
comp_store = pd.HDFStore(save_to)
comp_keys = comp_store.keys()
print('{} tables in compressed val'.format(len(comp_keys)))

In [ ]:
comp_store.select('d0')